In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ridership Open Lakehouse Demo (Part 2): Simulate Bus Rides using Apache Spark

This notebook will demonstrate a strategy to implement an open lakehouse on GCP, using Apache Iceberg,
as an open source standard for managing data, while still leveraging GCP native capabilities. This demo will use
BigQuery Manged Iceberg Tables, Managed Apache Kafka and Apache Kafka Connect to ingest streaming data, Vertex AI for Generative AI queries on top of the data and Dataplex to govern tables.

This notebook will process the data to simulate bus rides. The original datasets describes `ridership` as passengers in bus stations. We created bus lines, but not the rides. This will be the goal of this notebook. We accomplish this using **Google's Cloud Serverless Apache Spark**.

The processing will simulate `bus ridership` data, based on `bus station ridership` data. The `bus station ridership` shows passangers waiting at a given station at a given timestamp. Our PySpark processing pipelines will use Pandas UDFs, simulating a bus picking up those passangers while driving it's route. The routes for the buses are taken from the pre-made `bus_lines` table.

All data in this notebook was prepared in the previous `part0` notebook, and loaded in `part1` notebook.

## Setup environment

In [ ]:
USER_AGENT = "cloud-solutions/data-to-ai-nb-v3"

PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
BQ_DATASET = "ridership_lakehouse"
BUCKET_NAME = f"{PROJECT_ID}-ridership-lakehouse"
LOCATION = "us-central1"
BQ_CONNECTION_NAME = "cloud-resources-connection"
TABLE_NAME = "bus_rides"
EXTERNAL_WAREHOUSE_PREFIX = "external_iceberg_warehouse"
EXTERNAL_WAREHOUSE = f"gs://{BUCKET_NAME}/{EXTERNAL_WAREHOUSE_PREFIX}/"
BQ_WAREHOUSE_PREFIX = "iceberg_data"
BQ_WAREHOUSE = f"gs://{BUCKET_NAME}/{BQ_WAREHOUSE_PREFIX}/"

print(PROJECT_ID)
print(BUCKET_NAME)

In [ ]:
from google.cloud import bigquery, storage
from google.api_core.client_info import ClientInfo

# we will use the storage client only for demonstration purposes
storage_client = storage.Client(
    project=PROJECT_ID,
    client_info=ClientInfo(user_agent=USER_AGENT)
)

# we will use the bigquery client to prepare an empty table, backed by apache iceberg parquet format.
bigquery_client = bigquery.Client(
    project=PROJECT_ID,
    location=LOCATION,
    client_info=ClientInfo(user_agent=USER_AGENT)
)

In [ ]:
# Some helper functions

import pandas as pd

pd.set_option('display.max_colwidth', None)

def display_blobs_with_prefix(prefix: str, top=20):
  blobs = [[b.name, b.size, b.content_type, b.updated] for b in
         storage_client.list_blobs(BUCKET_NAME, prefix=prefix, )]
  df = pd.DataFrame(blobs, columns=["Name", "Size", "Content Type", "Updated"])
  return df.head(top)


In [ ]:
# we will use these imports in multiple paragraphs of this notebook

import pyspark.sql.connect.functions as f
import pyspark.sql.types as t
from pyspark.sql import Row

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession
from google.cloud.dataproc_v1 import Session

session = Session()


external_catalog = "external_catalog"
bq_catalog = "bq_catalog"

# here, we're setting the options for the external catalog, where the tables are managed by spark, and read-only for bq
session.runtime_config.properties[f"spark.sql.catalog.{external_catalog}"] = "org.apache.iceberg.spark.SparkCatalog"
session.runtime_config.properties[f"spark.sql.catalog.{external_catalog}.type"] = "hadoop"
session.runtime_config.properties[f"spark.sql.catalog.{external_catalog}.gcp_project"] = f"{PROJECT_ID}"
session.runtime_config.properties[f"spark.sql.catalog.{external_catalog}.gcp_location"] = f"{LOCATION}"
session.runtime_config.properties[f"spark.sql.catalog.{external_catalog}.warehouse"] = f"{EXTERNAL_WAREHOUSE}"

# here, we're setting the options for the bigquery catalog, where the tables are managed by bigquery, and read-only for spark
session.runtime_config.properties[f"spark.sql.catalog.{bq_catalog}"] = "org.apache.iceberg.spark.SparkCatalog"
session.runtime_config.properties[f"spark.sql.catalog.{bq_catalog}.type"] = "hadoop"
session.runtime_config.properties[f"spark.sql.catalog.{bq_catalog}.gcp_project"] = f"{PROJECT_ID}"
session.runtime_config.properties[f"spark.sql.catalog.{bq_catalog}.gcp_location"] = f"{LOCATION}"
session.runtime_config.properties[f"spark.sql.catalog.{bq_catalog}.warehouse"] = f"{BQ_WAREHOUSE}"


# Create the Spark session. This will take some time.
spark = (
   DataprocSparkSession.builder
     .appName("simulate-bus-rides")
     .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.9.2") \
     .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
     .dataprocSessionConfig(session)
     .getOrCreate()
)
spark.version

## Load data from BigQuery

For the BigQuery managed Iceberg tables, we can use 2 methods to read the data. One is the BigQuery standard way, which defers the reading of data to the BigQuery engine. This means that BigQuery slots are being used to read the data.

We can also use the standard Iceberg libraries to read the data directly from GCS, and only use the BigQuery metastore to read the metadata of the iceberg files. This will not use any BigQuery slots to read the data. To do that, we need to export the metadata of the files to an iceberg manifest files.

In the next paragraphs we will demonstrate both methods, including how to export metdata from BigQUery.

In [ ]:
# For the ridership table, we will read directly from Iceberg data.
# to do that, we first will export the metadata from BigQuery

# take a peak on the metadata folder before exporting the metadata from BigQuery
display_blobs_with_prefix("iceberg_data/ridership/metadata")

In [ ]:
# export the metadata
bigquery_client.query(f"EXPORT TABLE METADATA FROM {BQ_DATASET}.ridership").result()

# then we can view the metadata in GCS
display_blobs_with_prefix("iceberg_data/ridership/metadata")

In [ ]:
# Now read the ridership iceberg data, from the bq catalog
ridership_df = spark.table(f"{bq_catalog}.ridership")
ridership_df.show(5)
ridership_df.printSchema()

In [ ]:
# and read the bus_lines data from the external catalog
bus_lines_df = spark.table(f"{external_catalog}.bus_lines")
bus_lines_df.show(5)
bus_lines_df.printSchema()

In [ ]:
# To read the bus_stations data, we will use reading directly from bigquery, which uses the underlying bigquery-spark connector and uses bigquery slots to read
bus_stations_df = spark.read.format("bigquery").load(f"{PROJECT_ID}.{BQ_DATASET}.bus_stations")
bus_stations_df.show(5)
bus_stations_df.printSchema()

## Prepare bus data to be joined with stations ridership

Our initial `ridership` table refers to passengers waiting in bus stations, along with minute-by-minute timestamps.

In order to prepare bus trips, picking up passangers along the way, we need to understand what are the available time ranges that we can simulate bus riders, then start simulating trips, from station to station.

In [ ]:
# 1. Calculate min and max transit_timestamp for each station_id
station_time_summary_df = ridership_df.groupBy("station_id").agg(
    f.min("transit_timestamp").alias("min_station_timestamp"),
    f.max("transit_timestamp").alias("max_station_timestamp")
)
station_time_summary_df.show(5)

In [ ]:
# 2. Explode the 'stops' array in bus_lines_df to get each station_id on a new row
exploded_bus_stops_df = bus_lines_df.withColumn("station_id", f.explode("stops"))

# 3. Join the exploded bus_stops_df with the station_time_summary_df
#    Use a 'left' join to keep all bus lines, even if some of their stops have no ridership data
bus_lines_with_station_times = exploded_bus_stops_df.join(
    station_time_summary_df,
    on="station_id",
    how="left"
)

# 4. Aggregate back by bus_line_id to find the overall min and max timestamp across all its stops
# we will take the max from the minimum stations, ensuring we have data for all the stations in the route of the bus line
# and the reverse for the maximum stations (take the min out of the max group)
bus_line_overall_times_df = bus_lines_with_station_times.groupBy("bus_line_id").agg(
    # The maximum of the min_station_timestamp for all stops on the line
    f.max("min_station_timestamp").alias("min_transit_timestamp"),
    # The minimum of the max_station_timestamp for all stops on the line
    f.min("max_station_timestamp").alias("max_transit_timestamp")
)


# this will show us for each bus line, the time ranges in which we can simulate rides
bus_line_overall_times_df.show(10)

In [ ]:
# join back to the full bus line data, to get the full context of each bus line
bus_lines_with_min_max = bus_line_overall_times_df.join(bus_lines_df, on="bus_line_id")
bus_lines_with_min_max.show(10)

## Simulate bus trips - without passengers (for now)

In [ ]:
from typing import List, Tuple, TypeVar
from datetime import datetime, timedelta
import random

# A bus ride is a tuple of:
# str: bus_ride_id - will be a composite of the bus line and the start-time
# str: bus_line_id
# str: bus_size (S, M, L)
# int: seating_capacity
# int: standing_capacity
# int: total_capacity
# int: stop_id
# int: stor_index
# datetime: stop datetime
BusRide = Tuple[str, str, str, int, int, int, int, int, datetime]

# the equivilant pyspark structtype:
BusRideStructType = t.StructType([
    t.StructField("bus_ride_id", t.StringType()),
    t.StructField("bus_line_id", t.StringType()),
    t.StructField("bus_size", t.StringType()),
    t.StructField("seating_capacity", t.IntegerType()),
    t.StructField("standing_capacity", t.IntegerType()),
    t.StructField("total_capacity", t.IntegerType()),
    t.StructField("bus_stop_id", t.IntegerType()),
    t.StructField("bus_stop_index", t.IntegerType()),
    t.StructField("timestamp_at_stop", t.TimestampType())
])

BUS_SIZE_MAP = {
  "S": (20, 20),
  "M": (30, 25),
  "L": (40, 30),
}

# Returns a list of bus_rides - this is a standard python function
# it will be wrapped to be a spark UDF
# it will be called for each bus line, that we have now the global start
# time and the last time stamp for which we have data for
# the argument `row` is a row from the bus lines described in the last paragraph
def generate_ride_times(row: t.Row) -> List[BusRide]:
  # global start time and end time (roughly 4 years of data between these timestamps)
  start_time = row.min_transit_timestamp
  end_time = row.max_transit_timestamp
  rides = []
  trips_count = 0
  # this loop will execute once while we're in the bounds of the global start and end times
  # basically each time this outer while loop runs, we will continue generating trips
  # the inner for loop will execute for each stop in one trip
  while start_time < end_time:
    # new ride id
    new_bus_ride_id = ride_id = f"{row.bus_line_id}_{start_time.strftime('%Y-%m-%d_%H-%M-%S')}"

    # some parameters for this bus ride, choosing a size of bus
    bus_size = random.choice(list(BUS_SIZE_MAP.keys()))
    seating_capacity, standing_capacity = BUS_SIZE_MAP[bus_size]

    # this will cover all the stops in one ride, will be merged to the outer `rides` list
    one_trip_stops = []

    # stop time is the timestamp at which we arrived at a bus stop. we start a ride with the same ride time.
    stop_time = start_time

    # for each station / bus stop
    for i, stop in enumerate(row.stops):
      # if we are now past the global end time, just stop the loop
      if stop_time > end_time:
        break
      # generate one stop data
      one_trip_stops.append((
          new_bus_ride_id,
          row.bus_line_id,
          bus_size,
          seating_capacity,
          standing_capacity,
          seating_capacity + standing_capacity,
          stop,
          i+1,
          stop_time
        ))
      # move the stop_time to the next stop by small number of minutes
      stop_time = stop_time + timedelta(minutes=random.choice([1, 2, 3]))
      # End generating bus stop data, move the next station
    # finished all stops in a station - move one_trip_stops data to rides list, before it resets
    rides.extend(one_trip_stops)
    # increase trips_count
    trips_count += 1
    # move the start time to the next, but the number of the bus line frequency minutes.
    start_time = start_time + timedelta(minutes=trips_count*row.frequency_minutes)
  return rides

# wrap our function to be a spark udf
generate_ride_times_udf = f.udf(generate_ride_times, t.ArrayType(BusRideStructType))

# we now select the call to the UDF, passing in each row, and exploding the output, since it is returning a list/array
bus_rides = bus_lines_with_min_max.select(f.explode(generate_ride_times_udf(f.struct(*bus_lines_with_min_max.columns))).alias("rides")).select("rides.*")

bus_rides.show(10)

## Join with ridership data

This is a short section, to join the `ridership` df, allowing us to see how many passengers are in each station of a trip, by bus_stop and timestamp

In [ ]:
# join with the ridership data, by each station_id and the timestamp
bus_rides_with_ridership = bus_rides.alias("rides").join(
    ridership_df.alias("ridership"),
     (f.col("rides.bus_stop_id") == f.col("ridership.station_id")) & (f.col("rides.timestamp_at_stop") == f.col("ridership.transit_timestamp")),
    "inner"
)

# drop reduntant columns and rename `ridership` to `passengers_in_stop` for clarity
bus_rides_with_ridership = bus_rides_with_ridership.drop("transit_timestamp").drop("station_id").withColumnRenamed("ridership", "passengers_in_stop")
# cache this, as this will be used again shortly
bus_rides_with_ridership.cache()
bus_rides_with_ridership.show(10)

In [ ]:
bus_rides_with_ridership.printSchema()

## Aggregate passengers

This section will aggregate for each ride, passengers from each station, taking into considiration the passengers alighting (passengers leaving the bus), and bus capacity.

this means we will take the `bus_rides_with_ridership` df, and extend it to contain more data, specifcally about:
- `passengers_alighting`: passengers departing at each stop
- `passengers_boarding`: passengers onboarding at each stop (we can't necessrily fit all the people in a stop to our bus)
- `remaining_capacity`: after passengers have boarded and alighted, how much room do we still have on the bus
- `remaining_at_stop`: number of passengers who have no more room left and have to wait at the stop for the next bus
- `total_passengers`: number of passengers on the bus, after boarding and alighting.

In [ ]:
import pandas as pd
import numpy as np

# Define the schema for the output of the Pandas UDF
# It should include all existing columns from bus_rides_with_ridership
# plus the new 'passengers_alighting' and 'total_passengers' columns.
simulated_ride_output_schema = t.StructType(bus_rides_with_ridership.schema.fields + [
    t.StructField("passengers_alighting", t.LongType(), True),
    t.StructField("passengers_boarding", t.LongType(), True),
    t.StructField("remaining_capacity", t.LongType(), True),
    t.StructField("remaining_at_stop", t.LongType(), True),
    t.StructField("total_passengers", t.LongType(), True),
])

# this python function acccepts a dataframe, and returns the same.
# each call to the function will be of a dataframe represeting one trip for one bus
# it will generate and calculate all the above attributes,
# and return an extended dataframe with all of the relevant data
def calculate_bus_passengers(pdf: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the total number of passengers on the bus at each stop for a given bus ride.
    The number of passengers alighting is a random proportion of the boarding ridership,
    but cannot exceed the current number of passengers on the bus.
    this tracks to the logic that a central station, will have many people coming into the bus but also many people leaving the bus.
    note, that in a real-world scenario, this might be an overly simplistic heuristic, but our use case will be good enough.
    """
    import numpy as np

    # andle empty input DataFrame gracefully
    if pdf is None or pdf.empty:
      # Return an empty DataFrame with the exact schema expected by Spark
      # This prevents UnboundLocalError when a group is empty.
      empty_pdf_data = pd.DataFrame(columns=[c for c in simulated_ride_output_schema.fieldNames])
      return pd.DataFrame(empty_pdf_data)


    # Sort the DataFrame by bus_stop_index to ensure correct iterative calculation
    pdf = pdf.sort_values(by="bus_stop_index").reset_index(drop=True)

    # Initialize passengers on the bus at the start of this ride segment
    current_passengers_on_bus = 0
    alighting_passengers_list = []
    passengers_boarding_list = []
    total_passengers_list = []
    remainig_capacity_list = []
    remaining_at_stop_list = []

    # for each bus stop along the ride
    for index, row in pdf.iterrows():
        # Get passengers trying to board at this stop (passengers_in_stop), treating None as 0
        # not sure they all have room
        passengers_in_stop = row['passengers_in_stop'] if pd.notnull(row['passengers_in_stop']) else 0

        # Calculate random passengers alighting:
        # It's a random normal distrubution, propotional to the passengers in the station (central stations have lots of people coming on and off).
        potential_alighting = int(np.random.normal(loc=passengers_in_stop, scale=passengers_in_stop / 4))

        # Ensure alighting passengers do not exceed the number of passengers currently on the bus
        # and also ensure it's not negative.
        passengers_alighting = max(0, min(potential_alighting, current_passengers_on_bus))

        # update the number of passengers on the bus (this will change again soon with the number of boarding passengers)
        current_passengers_on_bus = current_passengers_on_bus - passengers_alighting

        # Calculate the number of passengers currently on the bus after alighting
        temporary_capacity = row['total_capacity'] - current_passengers_on_bus

        # actual passengers on bus is the lower number from our remaining capacity after boarding, and people at the stop
        # meaning, that we will take all the pasesengers waiting, if we can.
        # if the capacity is lower, some will be left waiting on the bus station.
        passengers_boarding = min(temporary_capacity, passengers_in_stop)
        remaining_at_stop = passengers_in_stop - passengers_boarding

        # Passengers after boarding at this stop
        current_passengers_on_bus = current_passengers_on_bus + passengers_boarding

        # Store the results for this row
        alighting_passengers_list.append(passengers_alighting)
        total_passengers_list.append(current_passengers_on_bus)
        passengers_boarding_list.append(passengers_boarding)
        remainig_capacity_list.append(row['total_capacity'] - current_passengers_on_bus)
        remaining_at_stop_list.append(remaining_at_stop)

    # Add the new columns to the pandas DataFrame
    pdf['passengers_alighting'] = alighting_passengers_list
    pdf['total_passengers'] = total_passengers_list
    pdf['passengers_boarding'] = passengers_boarding_list
    pdf['remaining_capacity'] = remainig_capacity_list
    pdf['remaining_at_stop'] = remaining_at_stop_list
    return pdf

# Apply the Pandas UDF to the bus_rides_with_ridership DataFrame
# Group by bus_ride_id to apply the simulation independently for each ride
bus_rides_with_riders_and_totals = bus_rides_with_ridership.groupBy("bus_ride_id").applyInPandas(
    calculate_bus_passengers, schema=simulated_ride_output_schema
)

bus_rides_with_riders_and_totals.show(10, truncate=False)
bus_rides_with_riders_and_totals.printSchema()

## Store results to bigquery

In [ ]:
# drop an existing table, if needed
bigquery_client.delete_table(f"{PROJECT_ID}.{BQ_DATASET}.{TABLE_NAME}", not_found_ok=True)

In [ ]:
bus_rides_uri = f"gs://{BUCKET_NAME}/iceberg_data/{TABLE_NAME}/"

fields = bus_rides_with_riders_and_totals.schema.fields
fields_sql = ""

for field in fields:
  type_name = field.dataType.typeName().replace("Type()", "").upper()
  if type_name == "LONG":
    type_name = "INT64"
  fields_sql += field.name + " " + type_name + ",\n"


query = f"""
CREATE TABLE {BQ_DATASET}.{TABLE_NAME}
(
  {fields_sql}
)
WITH CONNECTION `{PROJECT_ID}.{LOCATION}.{BQ_CONNECTION_NAME}`
OPTIONS (
  file_format = 'PARQUET',
  table_format = 'ICEBERG',
  storage_uri = '{bus_rides_uri}');
"""
bigquery_client.query(query).result()

In [ ]:
# save the bus_rides_with_riders_and_totals dataframe to bigquery, with iceberge format
bus_rides_with_riders_and_totals.write \
  .format("bigquery") \
  .mode("overwrite") \
  .option("table", f"{PROJECT_ID}.{BQ_DATASET}.{TABLE_NAME}") \
  .save()
